In [21]:
import pandas as pd
import re
import math

In [22]:
cm_df = pd.read_csv('test_cm_parse_dynasty.txt', sep = '\t', names=['image_url', 'category', 'name', 'variant', 'cm_url', 'collector_number', 'rarity', 'num_available', 'price', 'super_category', 'edition'])

cm_df.head()


,image_url,category,name,variant,cm_url,collector_number,rarity,num_available,price,super_category,edition
0,https://static.cardmarket.com/img/db79f13bfbb1...,DYN,Command and Conquer,Cold Foil,https://www.cardmarket.com/en/FleshAndBlood/Pr...,0,Fabled,3,895.0,Dynasty,First
1,https://static.cardmarket.com/img/5df3289d2372...,DYN,"Emperor, Dracai of Aesir",Cold Foil,https://www.cardmarket.com/en/FleshAndBlood/Pr...,1,Legendary,1,150.0,Dynasty,First
2,https://static.cardmarket.com/img/a42ba138ac42...,DYN,"Emperor, Dracai of Aesir",Rainbow Foil,https://www.cardmarket.com/en/FleshAndBlood/Pr...,1,Legendary,1,200.0,Dynasty,First
3,https://static.cardmarket.com/img/6b8a6138f58a...,DYN,Dust from the Golden Plains,Regular,https://www.cardmarket.com/en/FleshAndBlood/Pr...,2,Majestic,108,1.0,Dynasty,First
4,https://static.cardmarket.com/img/388aeed020a8...,DYN,Dust from the Golden Plains,Rainbow Foil,https://www.cardmarket.com/en/FleshAndBlood/Pr...,2,Majestic,14,4.0,Dynasty,First


In [23]:
DEFAULT_PRICING_MAPPING = {
    'Token': {
        'Regular': 0.2,
        'Cold Foil': 2,
    },
    'Common': {
        'Regular': 0.2,
        'Rainbow Foil': 0.5,
        'Cold Foil': 0.5,
        'Extended Art Rainbow Foil': 0.5,
        'Reverse Foil': 0.5
    },
    'Fabled': {
        'Rainbow Foil': 2,
        'Cold Foil': 2,
    },
    'Legendary': {
        'Rainbow Foil': 2,
        'Cold Foil': 2,
        'Regular': 2,
        'Cold Foil Golden': 2,
    },
    'Marvel': {
        'Marvel': 2,
    },
    'Promo': {
        'Regular': 1,
        'Rainbow Foil': 2,
        'Cold Foil': 2,
        'Extended Art Rainbow Foil': 2,
        'Alternate Art Rainbow Foil': 2,
        'Artist Proof': 2,
        'Cold Foil Golden': 2,
    },  
    'Rare': {
        'Regular': 0.5,
        'Rainbow Foil': 1,
        'Cold Foil': 1,
        'Extended Art Rainbow Foil': 2,
    },
    'Super Rare': {
        'Regular': 0.75,
        'Rainbow Foil': 1.5,
    },
    'Majestic': {
        'Regular': 1,
        'Rainbow Foil': 2,
        'Cold Foil': 2,
        'Extended Art Rainbow Foil': 2,
        'Alternate Art Rainbow Foil': 2,

    },
}

by_supercategory = cm_df.groupby(['super_category'])
option1_name = 'Variant'
option2_name = "Edition"
option3_name = ""
standardised_product_type = "Arts & Entertainment > Hobbies & Creative Arts > Collectibles > Collectible Trading Cards"
custom_product_type = ""
variant_grams = 2
variant_inventory_tracker = "shopify"
variant_fulfillment = "manual"
variant_requires_shipping= "TRUE"
variant_taxable = "FALSE"
variant_barcode = ""
image_position = "1"
gift_card = "FALSE"
#might want to improve these SEO bits in the future, title will default to title,desciption defaults to body.
SEO_title = ""
SEO_description = ""
google_product_category = ""
google_shopping_gender = ""
google_shopping_age_group = ""
google_shopping_MPN = ""
google_shopping_adwords_grouping = ""
google_shopping_adwords_labels = ""
google_shopping_condition = ""
google_shopping_custom_product = ""
google_shopping_custom_abel_0 = ""
google_shopping_custom_label_1 = ""
google_shopping_custom_label_2 = ""
google_shopping_custom_label_3 = ""
google_shopping_custom_label_4 = ""
variant_weight_unit = "g"
variant_tax_code = ""
cost_per_item = ""
price_international = ""
price_compare_international =""
status = "active"

import_rows = []
import_rows_minimal = []
for super_category, frame in by_supercategory:
    #i think this group by is wrong, no idea why collector numbers are breaking though...
    additional_tags = ""
    if 'Hero Deck' in super_category:
        additional_tags = additional_tags + ",Hero Decks"
    if 'Blitz Deck' in super_category:
        additional_tags = additional_tags + ",Blitz Decks"
    if 'Promo' in super_category:
        additional_tags = additional_tags + ",Promos"
    print(f"{super_category}")
    by_name = frame.groupby('name')
    for name, frame2 in by_name:
        by_collector_number = frame2.groupby('collector_number')
        for collector_number, frame3 in by_collector_number:
            first_row = True
            for index, row in frame3.iterrows():
                #print(f"{super_category} {name} {row['variant']}")
                #create parent product
                handle = f"{re.sub('[^0-9a-zA-Z]+', '-', name)}-{row['collector_number']}-{re.sub('[^0-9a-zA-Z]+', '-', super_category)}".replace('--', '-')
                title = f"{name} [{row['collector_number']}]" 
                body = ""
                vendor = "fleshandbloodcards"
                tags = f"{super_category}{additional_tags},{row['collector_number']},{row['rarity'] }"
                published = "TRUE"
                option1_value = row['variant']
                option2_value = row['edition']            
                option3_value = ""          

                variant_inventory_qty = 0
                variant_inventory_policy = "deny"
                if row['variant'] == "Regular":
                    if row['rarity'] == "Common":
                        variant_inventory_qty = 6
                        variant_inventory_policy = "continue"
            
                    if row['rarity'] == "Rare":
                        variant_inventory_qty = 6
            
                variant_price = row['price']
                if float(row['price']) < DEFAULT_PRICING_MAPPING[row['rarity']][option1_value]:
                    variant_price = DEFAULT_PRICING_MAPPING[row['rarity']][option1_value]
                elif math.isnan(row['price']):
                    variant_price = DEFAULT_PRICING_MAPPING[row['rarity']][option1_value]
            
                variant_compare_at_price = ""
                #if the first variant isnt regular then the parent image may be wrong, could get clever here
                img_src = row['image_url']
                image_alt_text = f"{super_category} - {name}"
                variant_image = row['image_url']
                variant_sku = f"{handle}-{re.sub('[^0-9a-zA-Z]+', '-',option1_value)}-{re.sub('[^0-9a-zA-Z]+', '-',option2_value)}"
 
                if first_row:
                    import_rows.append([handle, title, body, vendor, standardised_product_type, custom_product_type, tags, 
                                    published, option1_name, option1_value, option2_name, option2_value, option3_name, option3_value,
                                    variant_sku, variant_grams, variant_inventory_tracker, variant_inventory_qty,
                                    variant_inventory_policy, variant_fulfillment, variant_price, variant_compare_at_price,
                                    variant_requires_shipping, variant_taxable, variant_barcode, img_src, image_position, image_alt_text,
                                    gift_card, SEO_title, SEO_description, google_product_category, google_shopping_gender, 
                                    google_shopping_age_group, google_shopping_MPN, google_shopping_adwords_grouping,
                                    google_shopping_adwords_labels, google_shopping_condition, google_shopping_custom_product,
                                    google_shopping_custom_abel_0, google_shopping_custom_label_1, google_shopping_custom_label_2,
                                    google_shopping_custom_label_3, google_shopping_custom_label_4,
                                    "", variant_weight_unit, variant_tax_code, cost_per_item, price_international, price_compare_international, status])
                    import_rows_minimal.append([handle, title, body, vendor, standardised_product_type, custom_product_type,
                                    published, option1_name, option1_value, variant_grams, variant_inventory_qty,
                                    variant_inventory_policy, variant_fulfillment, variant_price,
                                    variant_requires_shipping, variant_taxable, img_src, image_position, image_alt_text,
                                    gift_card, "", variant_weight_unit, status])         
                else:
                # create child variant use variant image and leave
                    import_rows.append([handle, "", "", "", "", "", "", 
                                    "", "", option1_value, "", option2_value, option3_name, option3_value,
                                    variant_sku, variant_grams, variant_inventory_tracker, variant_inventory_qty,
                                    variant_inventory_policy, variant_fulfillment, variant_price, variant_compare_at_price,
                                    variant_requires_shipping, variant_taxable, variant_barcode, "", "", "", "", "", "", "", "", 
                                    "", "", "", "", "", "", "", "", "", "", "", "", variant_weight_unit, variant_tax_code, cost_per_item, price_international, 
                                    price_compare_international,""])
                    import_rows_minimal.append([handle, title, body, vendor, standardised_product_type, custom_product_type,
                                    published, option1_name, option1_value, variant_grams, variant_inventory_qty,
                                    variant_inventory_policy, variant_fulfillment, variant_price,
                                    variant_requires_shipping, variant_taxable, "", "", "",
                                    gift_card, variant_image, variant_weight_unit, status])
                
                first_row = False
            
        

Dynasty


In [24]:
import_full = pd.DataFrame(import_rows, columns=['Handle', 'Title', 'Body (HTML)', 'Vendor', 'Standardized Product Type',
                                  'Custom Product Type', 'Tags', 'Published', 'Option1 Name', 'Option1 Value',
                                  'Option2 Name' , 'Option2 Value', 'Option3 Name', 'Option3 Value',
                                  'Variant SKU', 'Variant Grams', 'Variant Inventory Tracker',
                                  'Variant Inventory Qty', 'Variant Inventory Policy', 'Variant Fulfillment Service',
                                  'Variant Price', 'Variant Compare At Price', 'Variant Requires Shipping',
                                  'Variant Taxable', 'Variant Barcode', 'Image Src', 'Image Position', 'Image Alt Text',
                                  'Gift Card', 'SEO Title', 'SEO Description', 'Google Shopping / Google Product Category',
                                  'Google Shopping / Gender', 'Google Shopping / Age Group', 'Google Shopping / MPN',
                                  'Google Shopping / AdWords Grouping', 'Google Shopping / AdWords Labels' ,
                                  'Google Shopping / Condition', 'Google Shopping / Custom Product',
                                  'Google Shopping / Custom Label 0', 'Google Shopping / Custom Label 1',
                                  'Google Shopping / Custom Label 2', 'Google Shopping / Custom Label 3',
                                  'Google Shopping / Custom Label 4', 'Variant Image', 'Variant Weight Unit',
                                  'Variant Tax Code', 'Cost per item', 'Price / International',
                                  'Compare At Price / International', 'Status'])
import_full.to_csv('shopify_import_dynasty.csv', index=False)

In [25]:
import_full.head()

,Handle,Title,Body (HTML),Vendor,Standardized Product Type,Custom Product Type,Tags,Published,Option1 Name,Option1 Value,...,Google Shopping / Custom Label 2,Google Shopping / Custom Label 3,Google Shopping / Custom Label 4,Variant Image,Variant Weight Unit,Variant Tax Code,Cost per item,Price / International,Compare At Price / International,Status
0,Aether-Quickening-Blue-199-Dynasty,Aether Quickening (Blue) [199],,fleshandbloodcards,Arts & Entertainment > Hobbies & Creative Arts...,,"Dynasty,199,Rare",TRUE,Variant,Rainbow Foil,...,,,,,g,,,,,active
1,Aether-Quickening-Blue-199-Dynasty,,,,,,,,,Regular,...,,,,,g,,,,,
2,Aether-Quickening-Red-197-Dynasty,Aether Quickening (Red) [197],,fleshandbloodcards,Arts & Entertainment > Hobbies & Creative Arts...,,"Dynasty,197,Rare",TRUE,Variant,Rainbow Foil,...,,,,,g,,,,,active
3,Aether-Quickening-Red-197-Dynasty,,,,,,,,,Regular,...,,,,,g,,,,,
4,Aether-Quickening-Yellow-198-Dynasty,Aether Quickening (Yellow) [198],,fleshandbloodcards,Arts & Entertainment > Hobbies & Creative Arts...,,"Dynasty,198,Rare",TRUE,Variant,Regular,...,,,,,g,,,,,active


In [10]:
import_full.groupby('Handle').count()

,Title,Body (HTML),Vendor,Standardized Product Type,Custom Product Type,Tags,Published,Option1 Name,Option1 Value,Option2 Name,...,Google Shopping / Custom Label 2,Google Shopping / Custom Label 3,Google Shopping / Custom Label 4,Variant Image,Variant Weight Unit,Variant Tax Code,Cost per item,Price / International,Compare At Price / International,Status
Handle,,,,,,,,,,,,,,,,,,,,,
Amethyst-Tiara-171-Dynasty,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Annihilate-the-Armed-Blue-135-Dynasty,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
Annihilate-the-Armed-Red-133-Dynasty,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
Annihilate-the-Armed-Yellow-134-Dynasty,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
Arakni-Huntsman-113-Dynasty,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
Blacktek-Whisperers-117-Dynasty,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
Blazen-Yoroi-45-Dynasty,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
Crown-of-Dominion-234-Dynasty,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Emperor-Dracai-of-Aesir-1-Dynasty,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
